# Import Libraries

In [ ]:
import pdfplumber
import re
import os
import pandas as pd
import logging

logging.getLogger("pdfminer").setLevel(logging.ERROR)

# Input Folder/Files

In [ ]:
FOLDER_PATH = r"C:/Users/LENOVO/Downloads/Kumpulan_CV"
OUTPUT_FILE = "Rekap_Data_Pelamar.xlsx"

# Set Keyword 

In [32]:
# Skill keyword
SKILL_DB = [
    "Python", "Java", "C++", "Go", "JavaScript", "TypeScript", "PHP", "SQL",
    "Machine Learning", "Deep Learning", "Data Analysis", "TensorFlow", "PyTorch", 
    "Pandas", "NumPy", "Scikit-learn", "NLP", "OpenCV", "Android", "Flutter",
    "React", "Vue", "Angular", "Node.js", "Django", "Flask", "FastAPI",
    "Git", "Docker", "Kubernetes", "AWS", "GCP", "Azure", "Excel", "Tableau"
]

# Keyword Kampus
UNI_KEYWORDS = [
    "UNIVERSITY", "UNIVERSITAS", "INSTITUTE", "INSTITUT", 
    "POLITEKNIK", "POLYTECHNIC", "ACADEMY", "AKADEMI", 
    "SCHOOL", "COLLEGE", "SEKOLAH"
]

# Keyword Blacklist (Biar pendidikan ga kemasukan deskripsi kerja)
BLACKLIST_WORDS = [
    "DELIVERED", "MANAGED", "DEVELOPED", "CREATED", "EXPERIENCE", "WORK",
    "PROJECT", "ACTIVITY", "ACTIVITIES", "SUPPORTING", "BUILT", "USING",
    "RESPONSIBLE", "INCLUDING", "THESIS", "SKILLS", "SUMMARY", 
    "COLLABORATED", "LED", "DESIGNED", "ACHIEVED", "AWARDED", "PRODUCTION"
]

✅ Setup Selesai! Target Folder: C:/Users/LENOVO/Downloads/Kumpulan_CV
📂 Ditemukan 2 file PDF.


# Logics for 

In [33]:
def get_name_by_size(first_page):
    """Mencari Nama berdasarkan ukuran font TERBESAR di Header"""
    try:
        width, height = first_page.width, first_page.height
        # Crop 25% atas
        header = first_page.within_bbox((0, 0, width, height * 0.25))
        chars = header.chars 
        
        if not chars: return "Unknown"
        
        # Cari ukuran font terbesar (abaikan spasi)
        valid_chars = [c for c in chars if c['text'].strip()]
        if not valid_chars: return "Unknown"
        
        max_size = max(c['size'] for c in valid_chars)
        
        # Ambil huruf yang ukurannya MAX tadi
        large_chars = [c for c in valid_chars if abs(c['size'] - max_size) < 0.5]
        extracted_name = "".join([c['text'] for c in large_chars])
        
        return re.sub(r'\s+', ' ', extracted_name).strip()
    except:
        return "Unknown"

def get_education_strict(text_full):
    """Mencari Kampus & IPK dengan filter ketat"""
    edu_list = []
    gpa_found = "-"
    
    # --- CARI IPK ---
    match_label = re.search(r'(?:GPA|IPK)\s*[:]?\s*(\d\.\d{1,2})', text_full, re.IGNORECASE)
    match_slash = re.search(r'(\d\.\d{1,2})\s*/\s*4\.00', text_full) # Pola 3.63/4.00

    val = 0.0
    if match_label: val = float(match_label.group(1))
    elif match_slash: val = float(match_slash.group(1))

    if 2.0 <= val <= 4.0: gpa_found = str(val)

    # --- CARI KAMPUS ---
    lines = text_full.split('\n')
    for line in lines:
        clean = line.strip()
        upper = clean.upper()
        
        # Filter Panjang & Blacklist
        if len(clean) < 4 or len(clean) > 60: continue 
        if any(k in upper for k in UNI_KEYWORDS):
            if any(bad in upper for bad in BLACKLIST_WORDS): continue
            
            campus, major = clean, ""
            
            if "|" in clean:
                parts = clean.split("|")
                if any(k in parts[0].upper() for k in UNI_KEYWORDS): campus, major = parts[0].strip(), parts[1].strip()
                else: campus, major = parts[1].strip(), parts[0].strip()
            elif " - " in clean:
                parts = clean.split(" - ")
                if any(k in parts[0].upper() for k in UNI_KEYWORDS): campus = parts[0].strip()
            
            if len(campus) > 6:
                entry = f"{campus} ({major})" if major and len(major) < 40 else campus
                if entry not in edu_list: edu_list.append(entry)

    return {"Kampus": "; ".join(edu_list) if edu_list else "-", "IPK": gpa_found}

print("✅ Fungsi Logika Siap!")

✅ Fungsi Logika Siap!


In [34]:
all_data_pelamar = []

if not os.path.exists(FOLDER_PATH):
    print("❌ Folder Error")
else:
    files = [f for f in os.listdir(FOLDER_PATH) if f.endswith(".pdf")]
    
    print(f"🚀 MEMULAI PROSES SCANNING ({len(files)} File)...\n")
    
    for i, filename in enumerate(files):
        pdf_path = os.path.join(FOLDER_PATH, filename)
        
        # --- VISUAL PEMISAH BIAR GAMPANG DIBACA ---
        print("-" * 60)
        print(f"[{i+1}/{len(files)}] Memproses: {filename}")
        
        try:
            with pdfplumber.open(pdf_path) as pdf:
                # 1. Ambil Nama (Logika Font Size)
                nama_found = get_name_by_size(pdf.pages[0])
                
                # 2. Ambil Teks Full (Semua Halaman)
                text_all = ""
                for page in pdf.pages:
                    txt = page.extract_text()
                    if txt: text_all += txt + "\n"
                
                # 3. Cari Kontak
                email = re.search(r'[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,10}', text_all)
                email_res = email.group(0) if email else "-"
                
                hp = re.search(r'[\+\(]?(?:62|0)[0-9\-\(\)\s]{8,20}', text_all)
                hp_res = hp.group(0).strip() if hp else "-"
                
                # 4. Cari Pendidikan (Logika Strict)
                edu_res = get_education_strict(text_all)
                
                # 5. Cari Skills
                text_lower = text_all.lower()
                found_skills = [s for s in SKILL_DB if re.search(r'\b' + re.escape(s.lower()) + r'\b', text_lower)]
                skills_res = ", ".join(list(set(found_skills))) if found_skills else "-"

                # --- PRINT HASILNYA DI LAYAR (LIVE CHECK) ---
                print(f"   👤 Nama   : {nama_found}")
                print(f"   🎓 Kampus : {edu_res['Kampus']}")
                print(f"   📊 IPK    : {edu_res['IPK']}")
                print(f"   🛠 Skills : {skills_res[:50]}...") # Print dikit aja biar ga menuhin layar
                
                # Simpan ke list data
                data_row = {
                    "Nama": nama_found,
                    "Email": email_res,
                    "HP": hp_res,
                    "Kampus": edu_res['Kampus'],
                    "IPK": edu_res['IPK'],
                    "Skills": skills_res,
                    "Nama File": filename
                }
                all_data_pelamar.append(data_row)
                
        except Exception as e:
            print(f"   ❌ ERROR membaca file ini: {e}")

    print("\n" + "="*60)
    print("✅ PROSES SELESAI!")

🚀 MEMULAI PROSES SCANNING (2 File)...

------------------------------------------------------------
[1/2] Memproses: CV Royyan oct 2025.pdf
   👤 Nama   : AhmadRoyyanFatah
   🎓 Kampus : Bangkit Academy (Google) (Machine Learning Cohort); Bandung Institute of Technology (ITB) (Physics); Bangkit Academy 2024
   📊 IPK    : -
   🛠 Skills : Machine Learning, Go, Python, Flutter, React, Data...
------------------------------------------------------------
[2/2] Memproses: Muhammad Fakhri Najmi Al-farobi-resume.pdf
   👤 Nama   : MUHAMMADFAKHRINAJMIAL-FAROBI
   🎓 Kampus : Institut Teknologi Bandung
   📊 IPK    : 3.63
   🛠 Skills : Tableau, SQL, Excel, Python...

✅ PROSES SELESAI!


# Import to Excel File

In [ ]:
if all_data_pelamar:
    df = pd.DataFrame(all_data_pelamar)
    
    # Urutkan Kolom
    cols = ["Nama", "Email", "HP", "Kampus", "IPK", "Skills", "Nama File"]
    df = df.reindex(columns=cols)
    
    output_path = os.path.join(FOLDER_PATH, OUTPUT_FILE)
    
    try:
        df.to_excel(output_path, index=False)
        print(f"✅ FILE TERSIMPAN: {output_path}")
        print("Buka file Excel untuk melihat hasil lengkapnya.")
    except PermissionError:
        print("❌ GAGAL MENYIMPAN! File Excel sedang terbuka. Tutup dulu aplikasinya!")
else:
    print("⚠️ Tidak ada data yang berhasil diekstrak.")